# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


---

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

---

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



#### Creation of table for query 1

In [8]:
query = """
        CREATE TABLE IF NOT EXISTS songinfo_session_item (
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (session_id, item_in_session)
        )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Set up of CSV file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songinfo_session_item (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to check for data entry and answer question 1

In [10]:
query = "SELECT song, artist, length FROM songinfo_session_item WHERE session_id = 338 AND item_in_session = 4"

try:
    rows = session.execute(query)
    for row in rows:
        print(list(row))
except Exception as e:
    print(e)

['Music Matters (Mark Knight Dub)', 'Faithless', 495.30731201171875]


'Music Matters (Mark Knight Dub)' by artist 'Faithless' with a length of 495.31 seconds is the song at sessionId=338 and itemInSession=4.

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Creation of table for query 2

In [11]:
query = """
        CREATE TABLE IF NOT EXISTS songinfo_user_session (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### Set up of CSV file

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songinfo_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT statement to check for data entry and answer question 2

In [13]:
query = "SELECT first_name, last_name, song, artist FROM songinfo_user_session WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
    for row in rows:
        print(list(row))
except Exception as e:
    print(e)

['Sylvie', 'Cruz', "Keep On Keepin' On", 'Down To The Bone']
['Sylvie', 'Cruz', 'Greece 2000', 'Three Drives']
['Sylvie', 'Cruz', 'Kilometer', 'Sebastien Tellier']
['Sylvie', 'Cruz', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Lonnie Gordon']


The user with the userId=10 is Sylvie Cruz. At sessionId=182, she listened to four songs: 'Keep On Keepin' On' by 'Down To The Bone', 'Greece 2000' by 'Three Drives', 'Kilometer' by 'Sebastien Tellier' and 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)' by 'Lonnie Gordon'.

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creation of table for query 3    

In [14]:
query = """
        CREATE TABLE IF NOT EXISTS songinfo_user_songtitle (
            song TEXT,
            user_id INT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY (song, user_id)
        )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set up of CSV file

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songinfo_user_songtitle (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT statement to check for data entry and answer question 3

In [16]:
query = "SELECT first_name, last_name FROM songinfo_user_songtitle WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
    for row in rows:
        print(list(row))
except Exception as e:
    print(e)

['Jacqueline', 'Lynch']
['Tegan', 'Levine']
['Sara', 'Johnson']


The users Jacqueline Lynch, Tegan Levine and Sara Johnson listened to the song 'All Hands Against His Own'. 

---

### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS songinfo_session_item"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
query = "DROP TABLE IF EXISTS songinfo_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "DROP TABLE IF EXISTS songinfo_user_songtitle"
try:
    session.execute(query)
except Exception as e:
    print(e)

---

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()